In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.environ.get('riot_api_key'))

#ethan was here

RGAPI-9fa7d444-3a4c-4719-ba76-0ac6901974b8


In [6]:
import pygsheets

service_account = pygsheets.authorize(service_account_file='JSONs\\spreadsheet-automator-449222-28b7c907226f.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1p3ecWHthVRXvAEr900K8YOEt_c2ey98v8ym3I_GTWEA/edit?usp=sharing')


In [7]:
test = sheet.worksheet('title', 'Test')

In [8]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests

api_key= os.environ.get('riot_api_key')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):

    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)
        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+api_key)
        return response.json()['puuid']

def get_idtag_from_puuid(puuid=None):
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key'+api_key)

    id = {
        'gameName' : response.json()['gameName'],
        'tagLine' : response.json()['tagLine']
    }

    return id

In [9]:
import pandas as pd

def get_ladder(top=None,):
    root = 'https://na1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    masters = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'
    chall_response = requests.get(root + chall + '?api_key=' + api_key)
    gm_response = requests.get(root + gm + '?api_key=' + api_key)
    masters_response = requests.get(root + masters + '?api_key=' + api_key)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, masters_df]).reset_index(drop=True)[0:top]
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] = ladder['rank'] + 1
    
    return ladder

In [15]:
get_ladder(2000)

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,U0NRKUez6uuiYSLieKIlT_LL25pd-vaA0kjdsIMMJLvjsZ8,1588,98,55,False,False,False,False
1,2,yKKXTvcFPlTL8qIFO8j7h5pIsihllh92VliC1m1IlO7IFcM,1498,116,79,True,False,False,False
2,3,zTDVfcGmuxQnHL5PIc0dfCIe6t_nAVgWMJOR0dCispnn1Zyg,1424,82,48,False,False,False,False
3,4,CGJvSeU4ZQWpJnxUL9I0h9RCAsqFpl5kUHbLGluse9XLQ-...,1369,83,43,False,False,False,False
4,5,jbMwvSqQeMSQaNYp8WTTIvElc1PLv2e61ied3LcI0kSF4Mg,1319,97,72,True,False,False,True
...,...,...,...,...,...,...,...,...,...
1995,1996,tDwxhbarwWIqTbexTCPCE7yUY0O4FoAudk0H_hRgTXO2rdA,55,47,58,False,False,True,True
1996,1997,M6briVt8-baskG_VK9CK0oD-_73Q2YElThowAWq--LPZvC0,55,53,45,False,False,False,False
1997,1998,JeqyxHlb0xOFBD4VxUXqhFIRCY6EeETl8xgtY-LDZ03_1CA,55,70,66,False,False,True,True
1998,1999,r_KXbthznjz1qPWK-TdJaEAPNxKFs4b5XNOdj_1g9I4FFwV-,54,40,45,False,False,True,False


In [16]:
get_puuid(gameName = '23ejd', tagLine = 'NA1')

'4ZGjIHENXAR9luY6bUk077Y9emLdwSalP6D-KenK5mbxKGoyWu0iQWa1a7IX4SdzaNl761curDFYEw'

In [12]:
def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'

    response = requests.get(root_url + endpoint + query_params +'&api_key=' + api_key)


    return response.json()

def get_match_data_from_id(region=None, matchId=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{matchId}'

    response = requests.get(root_url+endpoint+'?api_key='+api_key)

    return response.json()

In [13]:
get_match_data_from_id(region="americas", matchId='NA1_5218360162')

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_5218360162',
  'participants': ['4ZGjIHENXAR9luY6bUk077Y9emLdwSalP6D-KenK5mbxKGoyWu0iQWa1a7IX4SdzaNl761curDFYEw',
   'USL28zKU4OHApMzJ32yf8JUtVDy4iTBxnsPZhOsaRTgnXDmplMfHxAUBh72mDceqAKzyOgjGBOxyQQ',
   'ZIPDkB2Wy_-pF6BWn1ipwNWChmxrPRwIZI4nW2wjs86FGz6KP-K7jCx8Fycm3yOVB7VWVg-jYatnvw',
   'podsNUGPSvp7NFmwK5CN2b7-kuJiDeciSCveMXtuAM8K8qYkGt71g815t0R6x5LHgpTgtpSuFs1cBQ',
   'XQbgZrSaGzaT8DU7DNIi_HxqyA0C2bO2kvLEyPWU_MyzEtZFxUjoA11xZX8_piBeTKfgtlBimeOESA',
   'AGtpfsoyEUYs_aIQ7KRJIVC1ALDrimqFG_J45J3M9cujcYsg03hPUr81PHCtswPLYWWeCezpMHtR5A',
   'sJA1bi-6I3KXrXTjptKFeF57_xyIHnx_nLCvcMjeqmHHWF05_tLKkAtRP__oOEakyFKIEis24hdWfQ',
   'YsI3vLwPizGp6sUgKCuhn7HLLPXP3cnijiDAwR829KGm0j5rqYvc_NzqUl3cLdK8DTPWDaEmDCjUcw',
   'zAt8ZkB8gn3Y7Gzb2wQHaVpp0v11ROeMQuyDB3cX91zIfvZu5eywPDQ74oEZshDWHd3bW8uymwhFUA',
   'I-XioiATTXapZ2X4tPkqk-i9Aow_GJBwrYGZU3HirJv5pySCpsNPwfapIHvRXsE82TdlA9IKzhyVfA']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1738

In [14]:
get_match_history(region='americas', puuid=get_puuid(tagLine='NA1', gameName='23ejd'), start=0, count=100)

['NA1_5218360162',
 'NA1_5218324497',
 'NA1_5217659620',
 'NA1_5217643149',
 'NA1_5216011707',
 'NA1_5215369769',
 'NA1_5215333134',
 'NA1_5215287984',
 'NA1_5215247665',
 'NA1_5214956912',
 'NA1_5214933780',
 'NA1_5214918120',
 'NA1_5214531166',
 'NA1_5214506434',
 'NA1_5214142443',
 'NA1_5214109638',
 'NA1_5214097750',
 'NA1_5214003856',
 'NA1_5213553874',
 'NA1_5213521953',
 'NA1_5213490951',
 'NA1_5213460314',
 'NA1_5213443400',
 'NA1_5212771101',
 'NA1_5212746743',
 'NA1_5212634531',
 'NA1_5212603992',
 'NA1_5212579624',
 'NA1_5212176314',
 'NA1_5211489372',
 'NA1_5211460416',
 'NA1_5211104602',
 'NA1_5211090790',
 'NA1_5211067698',
 'NA1_5211050954',
 'NA1_5210692984',
 'NA1_5210669295',
 'NA1_5210602276',
 'NA1_5210568934',
 'NA1_5210548050',
 'NA1_5209956656',
 'NA1_5209938272',
 'NA1_5209919222',
 'NA1_5209900298',
 'NA1_5209878224',
 'NA1_5209862346',
 'NA1_5209721559',
 'NA1_5209697470',
 'NA1_5209672576',
 'NA1_5209632446',
 'NA1_5209566969',
 'NA1_5209550104',
 'NA1_520952